# CBF Tutorial - Part 1: Static Obstacles

Authors: Bardh, Tom and Shakiba

## Message

In this notebook, we present a short introduction on Control Barrier Functions (CBF). The keypoints are,
1. CBF can proof the barrier function in runtime.
2. CBF's structure is filter or observer of existing controller. Hence applying is flexsible and easy.

## Tutorial

First, let's start by defining a two-dimensional system, called **nimble_ant**, with the following dynamics:
$$
  \dot{x} = u = \begin{bmatrix}u_0 \\ u_1\end{bmatrix} \tag{1}\\
$$

Given an initial position $x^{init}$, our goal is to control this system to a goal location $x^{goal}$.
$$
x^{init} = \begin{bmatrix} x^{init}_0 \\  x^{init}_1 \end{bmatrix} \qquad	x^{goal} = \begin{bmatrix} x^{goal}_0 \\  x^{goal}_1 \end{bmatrix} \tag{2}\\
$$
However, the system should avoid certain areas in the workspace (more on this later).

One way to solve this problem is to pose it as an optimization problem. Specifically, we would like to solve the following problem:
$$
\min_{u} \quad  ||u_{ref} - u||\\
\textrm{s.t.} \quad  \text{CBF constraints to be added later} \tag{3}\\
$$
So, we will try to minimize the distance between the decission variable $u$ and reference controller $u_{ref}$, while satisfying some constraints. For now, let us disregard the constraint and focus on designing a reference controller.

## Reference controller

We will define a very simple reference controller as follows
$$
u_{ref} = k \cdot (x^{goal} - x) \tag{4}
$$
This is a two dimensional vector that takes the difference between the current system state (or location) and the goal location and multiply it with a gain factor $k$. This **simple** reference controller will take us from an initial location to the goal location. We can say it is a typical [proportional controller](https://en.wikipedia.org/wiki/Proportional_control). Next, we need to make sure that the system does not enter undesired states.

## Control Barrier Function (CBF) as Constraints

We assume that the undesired regions in the workspace can be defined with one or more [ellipses](https://en.wikipedia.org/wiki/Ellipse). In two dimensions, the region inside an ellypsis can be using the standard form with the following inequality
$$
\frac{z^2_0}{a^2} + \frac{z^2_1}{b^2}\leq 1 \tag{5}
$$
where $z_0,z_1$ define the origin and $a,b$ define the major, minor axes of the ellipse, respectively.

In our case, we would like to check whether our system is in the undesired region and therefore we modify the afforementined requirement
$$
B(x) = \frac{(x_0-z_0)^2}{a^2} + \frac{(x_1-z_1)^2}{b^2} - 1 \tag{6}
$$

CBFs are defined by the condition:
$$
\dot{B}(x) \ge -\alpha\cdot(B(x)) \tag{7}
$$

Where $\dot{B}(x) = \frac{\partial{B}}{\partial{t}}$ and $\alpha$ is a positive real value. In this example we set $\alpha=1$. $\dot{B}(x)$ indicates how the distance of the system to the barrier changes with respect to time. So, in our formulation  $B(x) < 0$ indicates whether the current state is in a unsafe set. For a more detailed explanation of Eq. (7) See Proposition 2 in "[Safety Verification of Hybrid Systems Using Barrier Certificates](http://web.mit.edu/~jadbabai/www/papers/hscc04_2.pdf)". In Eq. (7), $B(x)$ indicates the distance between the state of the system and the unsafe set. We need to ensure that the rate of change is always bigger than the distance to the bad set by a factor $\alpha$. In our case $\alpha$ is one but in other problems it can be different. If this inequality holds, then the system is forward invariant. It will not enter the bad set. Next, we need to find a way to introduce our control input into this equation so that we can ensure that this inequality remains true. We do so through by subsituting $\dot{B}(x)$ with an equivalent expression in terms of the control input.

In order to calculate $\dot{B}(x)$, we utilize the [chain rule](https://en.wikipedia.org/wiki/Chain_rule), which allows us to write
$$
\dot{B}(x) = \frac{\partial{B}}{\partial{x}} \cdot \frac{\partial{x}}{\partial{t}} \tag{8}
$$

Since $\frac{\partial{x}}{\partial{t}} = \dot{x}$, then
$$
\dot{B}(x) = \frac{\partial{B}}{\partial{x}} \cdot \dot{x} \tag{9}
$$
This we can easily solve by taking the derivative of $B(x)$ with respect to $x$
$$
\frac{\partial{B}}{\partial{x}} = \begin{bmatrix} \frac{2}{a^2}(x_0 - z_0) \\ \frac{2}{b^2}(x_1 - z_1)\end{bmatrix} \tag{10}
$$
(The expansion is shown in [01-DerivativeB.md](./01-DerivativeB.md))

Plugging this back in the original equation (7), since (9), (10), $\alpha=1$, we get
$$
\dot{B}(x) \ge -\alpha\cdot(B(x))\\
\equiv \frac{\partial{B}}{\partial{x}} \dot{x} \ge -\alpha\cdot(B(x))\\ \equiv \begin{bmatrix} \frac{2}{a^2}(x_0 - z_0)\\
\frac{2}{b^2}(x_1 - z_1)\end{bmatrix} \dot{x} \ge -(\frac{(x_0-z_0)^2}{a^2} + \frac{(x_1-z_1)^2}{b^2} - 1) \tag{11}
$$
This will be used as a constraint for one ellipse. If there are additional ellipses in the workspace, we can add more CBFs in a similar fashion.

## Posing the control problem as a quadratic program

We can now combine the cost function with the constraints to form an optimization problem.
$$
\min_{u} \quad ||u_{ref} - u||\\ \textrm{s.t.} \quad \begin{bmatrix} \frac{2}{a^2}(x_0 - z_0) \\ \frac{2}{b^2}(x_1 - z_1)\end{bmatrix} u \ge -(\frac{(x_0-z_0)^2}{a^2} + \frac{(x_1-z_1)^2}{b^2} - 1) \\ \tag{12}
$$

This problem can be posed as a [quadratic program](https://en.wikipedia.org/wiki/Quadratic_programming) of the form
$$
\min_{\mathbf{x}} \quad  \frac{1}{2}\mathbf{x}^T P\mathbf{x} + q^T\mathbf{x} \\
\textrm{s.t.} \quad  G\mathbf{x} \le h \tag{13}
$$
In our case, $\mathbf{x} = \dot{x}$ is the decision variable,
Where

$$
P = I_2 \tag{14}
$$

$$
q = \begin{bmatrix} -u_{ref0} \\  -u_{ref1} \end{bmatrix} \tag{15}
$$

$$
G = \begin{bmatrix} \frac{-2}{a^2}(x_0 - z_0) \\
    \frac{-2}{b^2}(x_1 - z_1)\end{bmatrix} \tag{16}
$$

$$
h = (\frac{(x_0-z_0)^2}{a^2} + \frac{(x_1-z_1)^2}{b^2} - 1) \tag{17}
$$
(The expansion is show in [01-CBF2Quadratic.md](./01-CBF2Quadratic.md))

Here, one of advantage of this methodology is that instance of controller is not needed, as we can see (15) referes only result of controller reference $u_{ref}$. In another word, the CBF can work as an observer in whole controll system.

## Code implementation

First, we import the necessary packages for our example.


In [1]:
%matplotlib inline
from sympy import symbols, Matrix, sin, cos, lambdify, exp, sqrt, log, diff, Mul, srepr
import matplotlib.animation as animation
from cbflib import cbf, cbf_utils, sys_and_ctrl
import cvxopt as cvxopt
import control as control
import matplotlib.pyplot as plt
from matplotlib import rc
import numpy as np
from sympy import symbols


[SymPy](https://www.sympy.org/en/index.html) is a Python library for symbolic mathematics.
We use it for handling any formulas in symbolic.

numpy and matplotlib are well known.

[control](https://python-control.readthedocs.io/en/0.8.4/) is Python Control Systems Library (python-control).
That is a Python package that implements basic operations for analysis and design of feedback control systems.
We are using it to make simple contoller.

[cvxopt](https://cvxopt.org/) is Python Software for Convex Optimization.

In cbflib, we define several functions. 

In [cbf.py](./cbflib/cbf.py) we define the CBF class, which enables us to define an instance of a CBF as well as compute the G and h matrices in the controller. 

In [sys_and_ctrl](./cbflib/sys_and_ctrl.py), we define systems and controllers for our examples including nimble_ant.

Next, we define the goal for the system, bad sets, and parameters for the controller



In [2]:
# Robot Goal
x_goal = np.array([5, 5])

# Undesired aread in ellipse format (x,y,rad_x,rad_y) - Use example(0) through example(3)
bad_sets = cbf_utils.example(1)

# Parameters for reference controller
ctrl_param = [0.3, 0.3]


We define the expression for the CBF, f(x) and g(x) for the dynamics of the system

In [3]:
# Symbols and equations for the CBF
x_0, x_1, z_0, z_1, a, b = symbols('x_0 x_1 z_0 z_1 a b')
symbs = (x_0, x_1, z_0, z_1, a, b)
# Barrier function - distance of robot to obstacle
B = ((x_0 - z_0)/a)**2 + ((x_1 - z_1)/b)**2 - 1

# dx = g(x)u - not used
f = 0
g = Matrix([0, 0, 1.0])

# Initialize CBF
my_CBF = cbf.CBF(B, f, g, states = (x_0, x_1), bad_sets = bad_sets, symbs = symbs, degree=1)


We also define simulation options and define the closed loop system using the control package. 

In [4]:
# Simulation settings
T_max = 20
n_samples = 200
T = np.linspace(0, T_max, n_samples)
dt = T[1]-T[0]
params = {'x_goal': x_goal, 'bad_sets': bad_sets,
          'ctrl_param': ctrl_param, 'CBF': my_CBF}


Next, we define the initial conditions for the system as follows

In [5]:
# intial condition
x_0 = np.array([0.5, 1.5])

# Disable cvxopt optimiztaion output
cvxopt.solvers.options['show_progress'] = False
cvxopt.solvers.options['max_iter'] = 1000

# Simulate system
print('\nComputing trajectories for the initial condition:')
print('x_0\t x_1')
print(x_0[0], '\t', x_0[1])



Computing trajectories for the initial condition:
x_0	 x_1
0.5 	 1.5


We simulate the system.

In [6]:
# If initial condition is inside the bad set, error out.
for idxj, j in enumerate(bad_sets):
    curr_bs = bad_sets[idxj]
    assert cbf_utils.is_inside_ellipse(
        [x_0[0], x_0[1]], bad_sets[idxj]) == 0, "Initial condition is inside ellipse"

# Compute output on the nimble ant system for given initial conditions and timesteps T
x = np.zeros((np.size(x_0), len(T)))
x[:, 0] = x_0

# Simulate system
for i in range(len(T)-1):
    x[:, i+1] = x[:, i] + dt * \
        np.array(sys_and_ctrl.nimble_ant_f(T[i], x[:, i], [], params))

print("\n*Simulation Done. See plot for animation...")



*Simulation Done. See plot for animation...


We animate the system.

In [7]:
# Animate

# Init Plot
fig, ax = plt.subplots(figsize=(8,8))
ax = cbf_utils.plot_cbf_elements(ax, bad_sets, x_goal)
plt.xlim(0, 6)
plt.ylim(0, 6)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

line1, = ax.plot([], [], lw=2)
goal_square = plt.Rectangle(x_goal-np.array([.1, .1]), .2, .2, color='r', alpha=0.5)

def animate(i):
    line1.set_data((x[0][:i], x[1][:i]))
    return line1,

plt.close()

ani = animation.FuncAnimation(
    fig, animate, interval=30, frames=n_samples, repeat=True, blit=True)


from IPython.display import HTML
from matplotlib import rcParams
rcParams['animation.html'] = 'html5'
HTML(ani.to_html5_video())
ani


To run the code directly in python see [01-Tutorial-CFB-Static-Obstacles.py](./01-Tutorial-CFB-Static-Obstacles.py)